In [1]:
import os
import re
import json
import pandas as pd

In [2]:
ans1 = pd.read_csv('data_zhihu/daily_experience/你日常的一天是怎么度过的.csv', index_col=0)['answer']

In [3]:
ans2 = pd.read_csv('data_zhihu/daily_experience/你的日常生活是什么样子的.csv', index_col=0)['answer']

In [4]:
vocabs = {}
vocabs_dir_path = './vocab'
topns = [25, 50, 75, 100]
thresholds = [0.7, 0.75, 0.8, 0.85]
vocab_names = ['new_vocab_{}_{}'.format(topn, thresh) for topn in topns for thresh in thresholds]
vocab_names.append('old_vocab')
for vocab_name in vocab_names:
    with open(os.path.join(vocabs_dir_path, vocab_name+'.dic'), 'r', encoding='utf-8') as f:
        vocab = {}
        for line in f.readlines()[83:]:
            x = line.strip().split('\t')
            vocab[x[0]] = x[1:]
        vocabs[vocab_name] = vocab

In [5]:
word_types = [
1, # function (Function Words)
	2, # pronoun (Pronouns)
		3, # ppron (Personal Pronouns)
			4, # i (I)
			5, # we (We)
			6, # you (You)
			7, # shehe (SheHe)
			8, # they (They)
			19, # youpl
		9, # ipron (Impersonal Pronouns)
	11, # prep (Prepositions)
	12, # auxverb (Auxiliary Verbs)
	13, # adverb (Adverbs)
	14, # conj (Conjunctions)
	15, # negate (Negations)
	17, # quanunit
	18, # prepend
	131, # specart
	132, # tensem
		90, # focuspast (Past Focus)
		91, # focuspresent (Present Focus)
		92, # focusfuture (Future Focus)
		133, # progm
	140, # particle
		141, # modal_pa
		142, # general_pa
# othergram (Other Grammar)
	22, # compare (Comparisons)
	23, # interrog (Interrogatives)
	24, # number (Numbers)
	25, # quant (Quantifiers)
30, # affect (Affect)
	31, # posemo (Positive Emotions)
	32, # negemo (Negative Emotions)
		33, # anx (Anx)
		34, # anger (Anger)
		35, # sad (Sad)
40, # social (Social)
	41, # family (Family)
	42, # friend (Friends)
	43, # female (Female)
	44, # male (Male)
50, # cogproc (Cognitive Processes)
	51, # insight (Insight)
	52, # cause (Causal)
	53, # discrep (Discrepancies)
	54, # tentat (Tentative)
	55, # certain (Certainty)
	56, # differ (Differentiation)
60, # percept (Perceptual Processes)
	61, # see (See)
	62, # hear (Hear)
	63, # feel (Feel)
70, # bio (Biological Processes)
	71, # body (Body)
	72, # health (Health)
	73, # sexual (Sexual)
	74, # ingest (Ingest)
80, # drives (Drives)
	81, # affiliation (Affiliation)
	82, # achieve (Achievement)
	83, # power (Power)
	84, # reward (Reward)
	85, # risk (Risk)
100, # relativ (Relativity)
	101, # motion (Motion)
	102, # space (Space)
	103, # time (Time)
# persconc (Personal Concerns)
	110, # work (Work)
	111, # leisure (Leisure)
	112, # home (Home)
	113, # money (Money)
	114, # relig (Religion)
	115, # death (Death)
120, # informal (Informal Language)
	121, # swear (Swear)
	122, # netspeak (Netspeak)
	123, # assent (Assent)
	124, # nonflu (Nonfluencies)
	125, # filler (Filler Words)
]

In [6]:
def get_word_counts(answers):
    word_counts = {}
    word_type_counts = {}
    for vocab_name in vocab_names:
        print('counting', vocab_name)
        vocab = vocabs[vocab_name]
        word_counts[vocab_name] = {}
        word_type_counts[vocab_name] = {}
        for i in range(len(answers)):
            text_name = 'text_{:02}'.format(i)
            print('{:02} '.format(i), end='')
            word_counts[vocab_name][text_name] = {}
            word_type_counts[vocab_name][text_name] = {}
            for word_type in word_types:
                word_type_counts[vocab_name][text_name][word_type] = 0
            text = answers[i]
            for word in vocab:
                word_to_check = word
                word_to_check = word_to_check.replace('*', '.')
                word_to_check = word_to_check.replace('(', '\(')
                word_to_check = word_to_check.replace(')', '\)')
                matches = re.findall(word_to_check, text)
                if len(matches) > 0:
                    word_counts[vocab_name][text_name][word] = len(matches)
                    for word_type in vocab[word]:
                        word_type_counts[vocab_name][text_name][int(word_type)] += len(matches)
        print()
    return word_counts, word_type_counts

In [7]:
word_counts_ans1, word_type_counts_ans1 = get_word_counts(ans1)

counting new_vocab_25_0.7
00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 26

00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 

00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 

In [8]:
word_counts_ans2, word_type_counts_ans2 = get_word_counts(ans2)

counting new_vocab_25_0.7
00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 
counting new_vocab_25_0.75
00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 
counting new_vocab_25_0.8
00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 
counting new_vocab_25_0.85
00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 
counting new_vocab_50_0.7
00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 

In [9]:
with open('./word_counts/control/word_counts_control_1.json', 'w') as f:
    f.write(json.dumps(word_counts_ans1, ensure_ascii=False, indent=2))
with open('./word_counts/control/word_type_counts_control_1.json', 'w') as f:
    f.write(json.dumps(word_type_counts_ans1, ensure_ascii=False, indent=2))
    
with open('./word_counts/control/word_counts_control_2.json', 'w') as f:
    f.write(json.dumps(word_counts_ans2, ensure_ascii=False, indent=2))
with open('./word_counts/control/word_type_counts_control_2.json', 'w') as f:
    f.write(json.dumps(word_type_counts_ans2, ensure_ascii=False, indent=2))

In [10]:
for vocab_name in vocab_names:
    df1 = pd.DataFrame(word_type_counts_ans1[vocab_name]).T
    df1.to_csv('./word_counts/control/{}_control_1.csv'.format(vocab_name))
    df2 = pd.DataFrame(word_type_counts_ans2[vocab_name]).T
    df2.to_csv('./word_counts/control/{}_control_2.csv'.format(vocab_name))